In [ ]:
import cv2 #Librería para procesamiento de imágenes
import numpy as np

# Inicializar la cámara
cap = cv2.VideoCapture(0) #Usa la cámara por defecto (0) para capturar video en tiempo real

# Función para detectar colores en HSV que recibe 3 parámetros 

def detectar_color(hsv, x, y):  #HSV: Una imagen en el espacio de color . X,Y : Coordenadas de un pixel
    pixel = hsv[y, x]  # Obtener el color en la posición central del contorno
    h, s, v = pixel  #Separa los 3 valores del pixel. h: Tono o matiz, s: Saturación, v: Valor o brillo
    print(f"HSV en ({x}, {y}): H={h}, S={s}, V={v}")  #Imprime los valores HSV del pixel

    if (0 <= h <= 10) or (160 <= h <= 180):   #Si el tono está entre **0 y 10**, o entre **160 y 180**, se considera **rojo**.
                                              #El rojo está en ambos extremos del círculo de matices HSV.
        return "Rojo"
    elif 30 <= h <= 90:
        return "Verde"
    elif 90 <= h <= 130:
        return "Azul"
    elif 20 <= h <= 30:
        return "Amarillo"
    else:
        return "Desconocido"
    
    
#     El modelo de color **HSV (Hue, Saturation, Value)** es ideal para este tipo de detección porque el **matiz (`H`) separa los colores de manera más intuitiva** que BGR.
#     Los rangos de color pueden variar dependiendo de:
#   - Iluminación.
#   - Calidad de la cámara.
#   - Material del objeto.

# Función para detectar formas que recibe un único parámetro
def detectar_forma(contorno): #Contorno: una figura cerrada detectada en la imagen, generalmente obtenida con `cv2.findContours`.
    
    approx = cv2.approxPolyDP(contorno, 0.02 * cv2.arcLength(contorno, True), True) #Simplifica el contorno usando el algoritmo de Douglas-Peucker
    #cv2.arcLength(contorno, True): calcula el perímetro del contorno.
    #0.02 * perímetro: es la precisión del ajuste. Cuanto menor el número, más fiel será el contorno simplificado.
    #El resultado (approx) es una lista de puntos que representan los vértices aproximados de la figura.


    vertices = len(approx)  #Cuenta cuántos vértices tiene el contorno simplificado.

    # Obtener el área y el perímetro
    area = cv2.contourArea(contorno) #Calcula el área de la figura encerrada por el contorno.
    perimetro = cv2.arcLength(contorno, True) #Calcula el **perímetro** (longitud del contorno cerrado).

    # Comprobar si es un círculo usando la relación área/perímetro
    if perimetro > 0:
        circularidad = 4 * np.pi * (area / (perimetro ** 2))#Calcula la circularidad usando la fórmula
        
        if circularidad >0.85:  # Un círculo tiene circularidad cercana a 1 mientras que valores más bajos corresponden a figuras más irregulares.
            return "Circulo"

    if vertices == 3:
        return "Triangulo"
    elif vertices == 4:
        return "Cuadrado"
    

while True:
    ret, frame = cap.read() #Captura un frame de la cámara. ret: Indica si la captura fue exitosa(true or false), frame: Contiene la imagen actual
    if not ret:
        break

   
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #Convierte la imagen de color BGR a escala de grises.
                                                    #Útil para procesamiento de bordes y desenfoque (más eficiente que en color).
                                                    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)    #Convierte el frame a formato HSV.

                                                    #Se usa para detectar el color de los objetos.

   
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)    #Aplica un desenfoque gaussiano para suavizar la imagen y reducir el ruido.

                                                   #Esto ayuda a que la detección de bordes sea más precisa.

    
    edges = cv2.Canny(blurred, 50, 150)            #Aplica el algoritmo de Canny para detectar bordes.

                                                   #50 y 150 son los umbrales inferior y superior.

   
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Encuentra los contornos (formas cerradas) en la imagen de bordes.

                                                                                  #cv2.RETR_TREE: recupera todos los contornos y la jerarquía.

                                                                                 #cv2.CHAIN_APPROX_SIMPLE: simplifica los contornos para ahorrar memoria.

    for contorno in contours:
        area = cv2.contourArea(contorno) #Calcula el área del contorno actual.
        if area > 500:  # Filtrar ruido y contornos pequeños
            x, y, w, h = cv2.boundingRect(contorno) #Calcula un rectángulo envolvente del contorno. #x, y: esquina superior izquierda. w, h: ancho y alto del rectángulo.
            centro_x, centro_y = x + w // 2, y + h // 2  # Punto central del objeto

            # Detectar forma y color
            forma = detectar_forma(contorno)  #Llama a la función que determina la forma geométrica del contorno (círculo, cuadrado, etc.).
            color = detectar_color(hsv, centro_x, centro_y) #Llama a la función que determina el color en el centro del objeto.

            # Dibujar el contorno y la etiqueta en la imagen
            cv2.drawContours(frame, [contorno], -1, (0, 255, 0), 2)  #Dibuja el contorno en la imagen original (frame) con color verde y grosor de línea 2.
            
            cv2.putText(frame, f"{forma} - {color}", (x, y - 10),   #Escribe un texto sobre la imagen indicando la forma y el color detectado.
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)   #La posición (x, y - 10) está un poco encima del objeto.

    # Mostrar la imagen procesada
    cv2.imshow("Detección de Formas y Colores", frame)  #Muestra el frame procesado en una ventana llamada "Detección de Formas y Colores".

    # Presiona 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()   # Cuando termina el bucle se libera la cámara y se cierran las ventanas:
cv2.destroyAllWindows()


HSV en (456, 406): H=107, S=116, V=127
HSV en (456, 407): H=108, S=116, V=128
HSV en (142, 332): H=23, S=155, V=122
HSV en (142, 332): H=23, S=155, V=122
HSV en (302, 329): H=177, S=139, V=108
HSV en (302, 329): H=177, S=139, V=108
HSV en (441, 276): H=21, S=136, V=116
HSV en (441, 276): H=21, S=136, V=116
HSV en (273, 422): H=23, S=156, V=141
HSV en (273, 422): H=23, S=156, V=141
HSV en (109, 303): H=23, S=136, V=154
HSV en (109, 303): H=23, S=136, V=154
HSV en (109, 302): H=23, S=136, V=154
HSV en (109, 302): H=23, S=136, V=154
HSV en (274, 419): H=23, S=154, V=142
HSV en (274, 419): H=23, S=154, V=142
HSV en (413, 382): H=108, S=87, V=150
HSV en (413, 382): H=108, S=87, V=150
HSV en (109, 302): H=24, S=132, V=154
HSV en (109, 302): H=24, S=132, V=154
HSV en (108, 302): H=23, S=132, V=155
HSV en (108, 302): H=23, S=132, V=155
HSV en (407, 249): H=23, S=134, V=148
HSV en (407, 249): H=23, S=134, V=148
HSV en (278, 420): H=24, S=164, V=138
HSV en (278, 420): H=24, S=164, V=138
HSV en (